# Tools for viewing domain

This is a simple example used to make some images for a proposal, but shows a few of the ways we can make fancy images.

In [1]:
%matplotlib osx
#%matplotlib inline

In [2]:
import logging
import numpy as np
import fiona.crs
from matplotlib import pyplot as plt
from matplotlib import patches as pltp
from matplotlib import collections as pltc
import matplotlib.colors

import workflow
import workflow.ui
import workflow.warp
import workflow.plot
import workflow.crs

workflow.ui.setup_logging(1,None)

In [3]:
# set the HUC we wish to image, in this case the Delaware River Basin
HUC = '0204'
crs = workflow.crs.from_epsg('4269')

logging.info("")
logging.info("Meshing HUC: {}".format(HUC))
logging.info("="*30)
logging.info('Target projection: "{}"'.format(crs))
    

2020-05-14 09:52:52,847 - root - INFO: 
2020-05-14 09:52:52,852 - root - INFO: Meshing HUC: 0204
2020-05-14 09:52:52,854 - root - INFO: ==============================
2020-05-14 09:52:52,856 - root - INFO: Target projection: "pyproj.Proj('+units=m +init=epsg:4269 ', preserve_units=True)"


In [4]:
# get the sources
sources = workflow.source_list.get_default_sources()
sources['hydrography'] = workflow.source_list.hydrography_sources['NHD Plus']

workflow.source_list.log_sources(sources)

2020-05-14 09:52:52,871 - root - INFO: Using sources:
2020-05-14 09:52:52,873 - root - INFO: --------------
2020-05-14 09:52:52,874 - root - INFO: HUC: National Watershed Boundary Dataset (WBD)
2020-05-14 09:52:52,876 - root - INFO: hydrography: National Hydrography Dataset Plus High Resolution (NHDPlus HR)
2020-05-14 09:52:52,879 - root - INFO: DEM: National Elevation Dataset (NED)
2020-05-14 09:52:52,887 - root - INFO: soil type: National Resources Conservation Service Soil Survey (NRCS Soils)
2020-05-14 09:52:52,890 - root - INFO: land cover: National Land Cover Database (NLCD) Layer: NLCD_2016_Land_Cover_L48
2020-05-14 09:52:52,896 - root - INFO: soil thickness: None
2020-05-14 09:52:52,898 - root - INFO: meteorology: DayMet 1km


In [5]:
# get HUC 8s in the Delaware
crs, huc = workflow.get_split_form_hucs(sources['HUC'], HUC, 8, crs=crs)

2020-05-14 09:52:52,907 - root - INFO: 
2020-05-14 09:52:52,908 - root - INFO: Preprocessing HUC
2020-05-14 09:52:52,909 - root - INFO: ------------------------------
2020-05-14 09:52:52,910 - root - INFO: Loading level 8 HUCs in 0204.
2020-05-14 09:52:52,912 - root - INFO: Using HUC file "/Users/uec/research/water/data/watershed-workflow/data-master/hydrography/WBD_02_GDB/WBD_02.gdb"
2020-05-14 09:52:53,249 - root - INFO:   found 17 HUCs.
2020-05-14 09:52:53,250 - root - INFO:   -- 02040206
2020-05-14 09:52:53,251 - root - INFO:   -- 02040301
2020-05-14 09:52:53,252 - root - INFO:   -- 02040302
2020-05-14 09:52:53,253 - root - INFO:   -- 02040105
2020-05-14 09:52:53,258 - root - INFO:   -- 02040201
2020-05-14 09:52:53,259 - root - INFO:   -- 02040202
2020-05-14 09:52:53,260 - root - INFO:   -- 02040102
2020-05-14 09:52:53,261 - root - INFO:   -- 02040103
2020-05-14 09:52:53,265 - root - INFO:   -- 02040106
2020-05-14 09:52:53,266 - root - INFO:   -- 02040203
2020-05-14 09:52:53,274 - 

In [6]:
# we want the main channel.  This was available in a shapefile
_, delriv = workflow.get_shapes('../data/hydrologic_units/others/Delaware/delriv/delawareriver.shp', -1, crs)


2020-05-14 09:52:58,174 - root - INFO: 
2020-05-14 09:52:58,175 - root - INFO: Preprocessing Shapes
2020-05-14 09:52:58,177 - root - INFO: ------------------------------
2020-05-14 09:52:58,178 - root - INFO: loading file: "../data/hydrologic_units/others/Delaware/delriv/delawareriver.shp"
/Users/uec/codes/anaconda/3/envs/ats_meshing_20190719/lib/python3.7/site-packages/fiona/collection.py:336: FionaDeprecationWarning: Collection slicing is deprecated and will be disabled in a future version.
  return self.session.__getitem__(item)


In [7]:
# bring them all together in a single plot
#
# Note, we have to manage the layer order in matplotlib to get these to appear correctly.  This uses the zorder
# option available in most matplotlib plotting methods.

fig = plt.figure(dpi=300, figsize=(3,5.5))
ax = workflow.plot.get_ax(crs, fig)

workflow.plot.basemap(crs, ax, '10m', land_kwargs={'zorder':0}, 
                      ocean_kwargs={'zorder':5}, state_kwargs={'linewidth':0.3, 'zorder':1, 'color':'k'})
workflow.plot.huc(huc.exterior(), crs, ax=ax, color='darkgray', linewidth=0.4, zorder=2)

huc8s_delbay = [h8 for h8 in huc.polygons() if h8.properties['Name'] == 'Delaware Bay']
huc8s_not_delbay = [h8 for h8 in huc.polygons() if h8.properties['Name'] != 'Delaware Bay']

workflow.plot.shply(huc8s_delbay, crs, ax=ax, edgecolor='k', facecolor='lightsteelblue', 
                    alpha=0.3, linewidth=0.2, zorder=3)
workflow.plot.shply(huc8s_not_delbay, crs, ax=ax, edgecolor='k', facecolor='darkseagreen', 
                    alpha=0.3, linewidth=0.2, zorder=4)

# note too many reaches, the plot is overwhelmed
#workflow.plot.shply(reaches, crs, ax=ax, color='cyan', linewidth=0.1, zorder=6) 
#workflow.plot.shply(delriv, crs, ax=ax, facecolor='b', linewidth=0., zorder=7)
plt.savefig('delaware_river_map')
plt.show()

In [8]:
# Neversink
neversink_huc = '0204010403'
crs, neversink = workflow.get_split_form_hucs(sources['HUC'], neversink_huc, 12, crs=crs)

2020-05-14 09:53:06,320 - root - INFO: 
2020-05-14 09:53:06,321 - root - INFO: Preprocessing HUC
2020-05-14 09:53:06,323 - root - INFO: ------------------------------
2020-05-14 09:53:06,324 - root - INFO: Loading level 12 HUCs in 0204010403.
2020-05-14 09:53:06,326 - root - INFO: Using HUC file "/Users/uec/research/water/data/watershed-workflow/data-master/hydrography/WBD_02_GDB/WBD_02.gdb"
2020-05-14 09:53:06,899 - root - INFO:   found 8 HUCs.
2020-05-14 09:53:06,900 - root - INFO:   -- 020401040301
2020-05-14 09:53:06,901 - root - INFO:   -- 020401040302
2020-05-14 09:53:06,902 - root - INFO:   -- 020401040303
2020-05-14 09:53:06,903 - root - INFO:   -- 020401040304
2020-05-14 09:53:06,903 - root - INFO:   -- 020401040305
2020-05-14 09:53:06,904 - root - INFO:   -- 020401040306
2020-05-14 09:53:06,905 - root - INFO:   -- 020401040307
2020-05-14 09:53:06,906 - root - INFO:   -- 020401040308


In [9]:
# bring them all together in a single plot
#
# Note, we have to manage the layer order in matplotlib to get these to appear correctly.  This uses the zorder
# option available in most matplotlib plotting methods.

fig = plt.figure(dpi=300, figsize=(3,5.5))
ax = workflow.plot.get_ax(crs, fig)

workflow.plot.basemap(crs, ax, '10m', land_kwargs={'zorder':0}, 
                      ocean_kwargs={'zorder':5}, state_kwargs={'linewidth':0.3, 'zorder':1, 'color':'k'})
workflow.plot.huc(huc.exterior(), crs, ax=ax, color='darkgray', linewidth=0.4, zorder=2)

huc8s_delbay = [h8 for h8 in huc.polygons() if h8.properties['Name'] == 'Delaware Bay']
huc8s_not_delbay = [h8 for h8 in huc.polygons() if h8.properties['Name'] != 'Delaware Bay']

workflow.plot.shply(huc8s_delbay, crs, ax=ax, edgecolor='k', facecolor='lightsteelblue', 
                    alpha=0.3, linewidth=0.2, zorder=3)
workflow.plot.shply(huc8s_not_delbay, crs, ax=ax, edgecolor='k', facecolor='darkseagreen', 
                    alpha=0.3, linewidth=0.2, zorder=4)

workflow.plot.shply([neversink.exterior(),], crs, ax=ax, color='r', linewidth=0.3, zorder=5)
plt.savefig('delaware_river_map_ns')
plt.show()



In [13]:
# download/collect the river network within that shape's bounds
_, reaches = workflow.get_reaches(sources['hydrography'], '0204010403', 
                                  neversink.exterior().bounds, crs, long=0.06)
# simplify and prune rivers not IN the shape, constructing a tree-like data structure
# for the river network
reaches = workflow.hydrography.filter_rivers_to_shape(neversink.exterior(), reaches, 0.001)
#rivers = workflow.simplify_and_prune(neversink, reaches, simplify=10, cut_intersections=True)


2020-05-14 09:55:09,399 - root - INFO: 
2020-05-14 09:55:09,400 - root - INFO: Preprocessing Hydrography
2020-05-14 09:55:09,402 - root - INFO: ------------------------------
2020-05-14 09:55:09,403 - root - INFO: Loading streams in HUC 0204010403
2020-05-14 09:55:09,404 - root - INFO:          and/or bounds (-74.75507860169711, 41.30338049109724, -74.35742150648105, 42.03506939725321)
2020-05-14 09:55:09,407 - root - INFO: Using Hydrography file "/Users/uec/research/water/data/watershed-workflow/data-master/hydrography/NHDPlus_H_0204_GDB/NHDPlus_H_0204.gdb"
2020-05-14 09:55:14,680 - root - INFO:   found 4434 reaches
2020-05-14 09:55:15,035 - root - INFO:   filtered 5 of 2432 due to length criteria 0.06
2020-05-14 09:55:15,079 - root - INFO:   ...filtering


In [16]:
# get DEM
p, dem = workflow.get_raster_on_shape(sources['DEM'], neversink.exterior(), crs, mask=True, nodata=np.nan)


2020-05-14 09:56:05,637 - root - INFO: 
2020-05-14 09:56:05,638 - root - INFO: Preprocessing Raster
2020-05-14 09:56:05,640 - root - INFO: ------------------------------
2020-05-14 09:56:05,645 - root - INFO: collecting raster
2020-05-14 09:56:05,646 - root - INFO: Collecting DEMs to tile bounds: [-74.76507860169711, 41.293380491097246, -74.34742150648104, 42.04506939725321]
2020-05-14 09:56:05,648 - root - INFO:   Need:
2020-05-14 09:56:05,649 - root - INFO:     /Users/uec/research/water/data/watershed-workflow/data-master/dem/USGS_NED_1as_n42_w075.img
2020-05-14 09:56:05,652 - root - INFO:     /Users/uec/research/water/data/watershed-workflow/data-master/dem/USGS_NED_1as_n43_w075.img
2020-05-14 09:56:05,939 - root - INFO:  raster bounds = (-74.76507860169711, 42.04506939725321, -74.34730082390547, 41.2931249527838)


In [27]:
# just the neversink
fig = plt.figure(dpi=300, figsize=(3,5.5))
ax = workflow.plot.get_ax(crs, fig)
#workflow.plot.basemap(crs, ax, '10m', land_kwargs={'zorder':0}, 
#                      ocean_kwargs={'zorder':5}, state_kwargs={'linewidth':0.3, 'zorder':1, 'color':'k'})
workflow.plot.dem(p, dem, ax=ax)
workflow.plot.hucs(neversink, crs, ax=ax, color='r', linewidth=0.4, zorder=5)
workflow.plot.shply(reaches, crs, ax=ax, color='white', linewidth=0.3, zorder=4)
print(ax.get_xlim(), ax.get_ylim())
#ax.set_xlim(-77,-74.2)
#ax.set_ylim(39,42.4)

plt.savefig('neversink')
plt.show()

2020-05-14 10:15:16,591 - root - INFO: BOUNDS: (-74.76507860169711, 41.2931249527838, -74.34730082390547, 42.04506939725321)


(-74.76507860169711, -74.34730082390547) (41.2931249527838, 42.04506939725321)
